# Motivated by Brian Sack's ***Using Treasury STRIPS to Measure the Yield Curve***


https://www.federalreserve.gov/pubs/feds/2000/200042/200042pap.pdf

In [126]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, parent_dir)

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

from CurveDataFetcher import CurveDataFetcher
from CurveInterpolator import GeneralCurveInterpolator

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, no_logs_plz=True)

In [5]:
as_of_date = datetime(2024, 8, 29)
quote_type = "eod"

curve_set_df = curve_data_fetcher.build_curve_set(
    as_of_date=as_of_date,
    sorted=True, 
    use_github=True, 
)

In [6]:
pstrips_dict_df = curve_data_fetcher.public_dotcom_timeseries_api(
    cusips=curve_set_df[curve_set_df["security_type"] != "Bill"][
        "corpus_cusip"
    ].to_list(),
    start_date=as_of_date,
    end_date=as_of_date,
    max_concurrent_tasks=32,
)
pstrips_df = pd.DataFrame(
    [
        {
            "corpus_cusip": cusip,
            "pstrip_price": df.iloc[-1]["Price"],
            "pstrip_ytm": df.iloc[-1]["YTM"],
        }
        for cusip, df in pstrips_dict_df.items()
        if not df.empty
    ]
)
pstrips_df = pd.merge(
    curve_set_df[
        [
            "cusip",
            "label",
            "issue_date",
            "maturity_date",
            "time_to_maturity",
            "corpus_cusip",
        ]
    ],
    right=pstrips_df,
    on="corpus_cusip",
)

pstrips_df = pstrips_df[pstrips_df["time_to_maturity"] > 2/365]
pstrips_df

,cusip,label,issue_date,maturity_date,time_to_maturity,corpus_cusip,pstrip_price,pstrip_ytm
1,91282CFN6,"Sep 24s, 2-Year",2022-09-30,2024-09-30,0.087671,912821JT3,99.6045,4.687815
2,91282CFQ9,"Oct 24s, 2-Year",2022-10-31,2024-10-31,0.172603,912821JY2,99.2319,4.569212
3,91282CFX4,"Nov 24s, 2-Year",2022-11-30,2024-11-30,0.254795,912821KD6,98.8663,4.561705
4,91282CGD7,"Dec 24s, 2-Year",2023-01-03,2024-12-31,0.339726,912821KH7,98.5092,4.502871
5,91282CGG0,"Jan 25s, 2-Year",2023-01-31,2025-01-31,0.424658,912821KN4,98.1556,4.465770
...,...,...,...,...,...,...,...,...
214,912810SX7,"May 51s, 30-Year",2021-07-15,2051-05-15,26.726027,912803FY4,32.6239,4.283344
215,912810SZ2,"Aug 51s, 30-Year",2021-10-15,2051-08-15,26.978082,912803GA5,32.4270,4.265868
216,912810TD0,"Feb 52s, 30-Year",2022-04-18,2052-02-15,27.482192,912803GE7,31.9849,4.238236
217,912810TJ7,"Aug 52s, 30-Year",2022-10-17,2052-08-15,27.980822,912803GK3,31.5925,4.207328


In [133]:
cstrips_curve_set_df = pd.read_excel(r"C:\Users\chris\Curvy-CUSIPs\data\cstrips_info.xlsx")
cstrips_curve_set_df["time_to_maturity"] = (cstrips_curve_set_df["maturity_date"] - as_of_date).dt.days / 365

cstrips_dict_df = curve_data_fetcher.public_dotcom_timeseries_api(
    cusips=cstrips_curve_set_df["cusip"].to_list(),
    start_date=as_of_date,
    end_date=as_of_date,
    max_concurrent_tasks=32,
)
cstrips_quote_df = pd.DataFrame(
    [
        {
            "cusip": cusip,
            "cstrip_price": df.iloc[-1]["Price"],
            "cstrip_ytm": df.iloc[-1]["YTM"],
        }
        for cusip, df in cstrips_dict_df.items()
        if not df.empty
    ]
)
cstrips_quote_df = pd.merge(
    cstrips_curve_set_df[
        [
            "cusip",
            "label",
            "maturity_date",
            "time_to_maturity",
        ]
    ],
    right=cstrips_quote_df,
    on="cusip",
)
cstrips_quote_df

,cusip,label,maturity_date,time_to_maturity,cstrip_price,cstrip_ytm
0,912833LT5,"Nov 24, UST STRIPPED INT PMT",2024-11-15,0.213699,98.9632,4.979738
1,912833LU2,"Feb 25, UST STRIPPED INT PMT",2025-02-15,0.465753,97.9991,4.421774
2,912833LV0,"May 25, UST STRIPPED INT PMT",2025-05-15,0.709589,96.9464,4.456016
3,912833LW8,"Aug 25, UST STRIPPED INT PMT",2025-08-15,0.961644,96.0307,4.310477
4,912833LX6,"Nov 25, UST STRIPPED INT PMT",2025-11-15,1.213699,95.1620,4.182046
...,...,...,...,...,...,...
108,912834A65,"Nov 51, UST STRIPPED INT PMT",2051-11-15,27.230137,31.0869,4.387399
109,912834B56,"Feb 52, UST STRIPPED INT PMT",2052-02-15,27.482192,31.1078,4.343839
110,912834C55,"May 52, UST STRIPPED INT PMT",2052-05-15,27.728767,30.9668,4.321705
111,912834D47,"Aug 52, UST STRIPPED INT PMT",2052-08-15,27.980822,30.7939,4.302804


In [8]:
tbill_curve_set_df = curve_set_df[
    (curve_set_df["security_type"] == "Bill")
    & (curve_set_df["time_to_maturity"] > 5 / 360)
]

tbill_quotes = curve_data_fetcher.public_dotcom_timeseries_api(
    cusips=tbill_curve_set_df["cusip"].to_list(),
    start_date=as_of_date,
    end_date=as_of_date,
)

tbill_quotes_df = pd.DataFrame(
    [
        {
            "cusip": cusip,
            "tbill_price": df.iloc[-1]["Price"],
            "tbill_ytm": df.iloc[-1]["YTM"],
        }
        for cusip, df in tbill_quotes.items()
        if not df.empty
    ]
)
tbill_quotes_df = pd.merge(
    tbill_curve_set_df[
        [
            "cusip",
            "label",
            "maturity_date",
            "time_to_maturity",
        ]
    ],
    right=tbill_quotes_df,
    on="cusip",
)
tbill_quotes_df

,cusip,label,maturity_date,time_to_maturity,tbill_price,tbill_ytm
0,912797LG0,"Sep 24s, 17-Week",2024-09-10,0.032877,99.8310,5.530916
1,912797LH8,"Sep 24s, 17-Week",2024-09-17,0.052055,99.7096,5.806853
2,912797LJ4,"Sep 24s, 17-Week",2024-09-24,0.071233,99.6094,5.624514
3,912797LK1,"Oct 24s, 17-Week",2024-10-01,0.090411,99.5137,5.470692
4,912797LS4,"Oct 24s, 17-Week",2024-10-08,0.109589,99.4159,5.391075
5,912797LT2,"Oct 24s, 17-Week",2024-10-15,0.128767,99.3213,5.310998
6,912797LU9,"Oct 24s, 17-Week",2024-10-22,0.147945,99.2217,5.286430
7,912797LV7,"Oct 24s, 17-Week",2024-10-29,0.167123,99.1254,5.247039
8,912797MA2,"Nov 24s, 17-Week",2024-11-05,0.186301,99.0377,5.170517
9,912797MB0,"Nov 24s, 17-Week",2024-11-12,0.205479,98.9472,5.121420


In [149]:
fig = px.scatter(
    pstrips_df,
    x="time_to_maturity",
    y="pstrip_ytm",
    hover_data=["label", "corpus_cusip"],
    labels={"time_to_maturity": "Time to Maturity (Years)", "pstrip_ytm": "YTM"},
)
fig.update_traces(
    marker=dict(
        color="blue",
    ),
    name="PSTRIPS",
    showlegend=True,
)

fig2 = px.scatter(
    cstrips_quote_df,
    x="time_to_maturity",
    y="cstrip_ytm",
    hover_data=["label", "cusip"],
    labels={"time_to_maturity": "Time to Maturity (Years)", "cstrip_ytm": "YTM"},
)
fig2.update_traces(
    marker=dict(
        color="red",
    ),
    name="CSTRIPS",
    showlegend=True,
)

fig3 = px.scatter(
    tbill_quotes_df,
    x="time_to_maturity",
    y="tbill_ytm",
    hover_data=["label", "cusip"],
    labels={
        "time_to_maturity": "Time to Maturity (Years)",
        "tbill_ytm": "YTM",
    },
)
fig3.update_traces(
    marker=dict(
        color="green",
    ),
    name="Bills",
    showlegend=True,
)

for fig_data in [fig2.data, fig3.data]:
    for trace in fig_data:
        fig.add_trace(trace)


fig.update_layout(
    title=f"Market Observed US Treasury Zero Coupons YTMs - As of: {as_of_date.date()}",
    legend_title_text="Label",
    xaxis_title="Time to Maturity (Years)",
    yaxis_title=f"YTMs: {quote_type}_yield",
    width=1400,
    height=800,
    template="plotly_dark",
)

fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikethickness=1)
fig.show()

In [112]:
market_observed_zeros_curve_set_df = pd.concat(
    [
        tbill_quotes_df.rename(
            columns={"tbill_price": "obsv_price", "tbill_ytm": "obsv_ytm"}
        ),
        cstrips_quote_df.rename(
            columns={"cstrip_price": "obsv_price", "cstrip_ytm": "obsv_ytm"}
        ),
    ]
)
market_observed_zeros_curve_set_df = market_observed_zeros_curve_set_df.sort_values(
    by=["time_to_maturity"]
).drop_duplicates(subset=["time_to_maturity"])
# )
market_observed_zeros_curve_set_df = market_observed_zeros_curve_set_df[
    market_observed_zeros_curve_set_df["time_to_maturity"] > 30 / 360
]
market_observed_zeros_curve_set_df

,cusip,label,maturity_date,time_to_maturity,obsv_price,obsv_ytm
3,912797LK1,"Oct 24s, 17-Week",2024-10-01,0.090411,99.5137,5.470692
31,912797GW1,"Oct 24s, 52-Week",2024-10-03,0.095890,99.4869,5.432636
4,912797LS4,"Oct 24s, 17-Week",2024-10-08,0.109589,99.4159,5.391075
16,912797KT3,"Oct 24s, 26-Week",2024-10-10,0.115068,99.3879,5.373933
5,912797LT2,"Oct 24s, 17-Week",2024-10-15,0.128767,99.3213,5.310998
...,...,...,...,...,...,...
108,912834A65,U S TREAS SEC STRIPPED INT PMT 0.000...,2051-11-15,27.230137,31.0869,4.387399
109,912834B56,U S TREAS SEC STRIPPED INT PMT 0.000...,2052-02-15,27.482192,31.1078,4.343839
110,912834C55,U S TREAS SEC STRIPPED INT PMT 0.000...,2052-05-15,27.728767,30.9668,4.321705
111,912834D47,U S TREAS SEC STRIPPED INT PMT 0.000...,2052-08-15,27.980822,30.7939,4.302804


In [113]:
ttm_interp = np.linspace(0, 30, 1000)
x = market_observed_zeros_curve_set_df["time_to_maturity"].to_numpy()
y = market_observed_zeros_curve_set_df["obsv_ytm"].to_numpy() 
curve_interpolator = GeneralCurveInterpolator(
    x=x,
    y=y,
    linspace_x_lower_bound=0,
    linspace_x_upper_bound=30,
    linspace_x_num=100,
    enable_extrapolate_left_fill=True,
    enable_extrapolate_right_fill=True,
)

In [125]:
fig = px.scatter(
    cstrips_quote_df,
    x="time_to_maturity",
    y="cstrip_ytm",
    hover_data=["label", "cusip"],
    labels={"time_to_maturity": "Time to Maturity (Years)", "cstrip_ytm": "YTM"},
)
fig.update_traces(
    marker=dict(
        color="red",
    ),
    name="CSTRIPS",
    showlegend=True,
)

fig2 = px.scatter(
    tbill_quotes_df,
    x="time_to_maturity",
    y="tbill_ytm",
    hover_data=["label", "cusip"],
    labels={
        "time_to_maturity": "Time to Maturity (Years)",
        "tbill_ytm": "YTM",
    },
)
fig2.update_traces(
    marker=dict(
        color="green",
    ),
    name="Bills",
    showlegend=True,
)
for trace in fig2.data:
    fig.add_trace(trace)



fig.add_trace(
    go.Scatter(
        x=ttm_interp,
        y=curve_interpolator.univariate_spline(s=0.33, return_func=True)(ttm_interp),
        mode="lines",

        line=dict(
            color="blue",

        ),
        name="UniSpline (Cubic)",
    )
)
fig.add_trace(
    go.Scatter(
        x=ttm_interp,
        y=curve_interpolator.b_spline_with_knots_interpolation(
            knots=[0.25, 0.33, 0.5, 1, 2, 3, 5, 7, 10, 20], k=3, return_func=True
        )(ttm_interp),
        mode="lines",
        line=dict(
            color="yellow",
        ),
        name="BSpline 1",
    )
)

fig.update_layout(
    title=f"Market Observed Bill & C-STRIP YTMs - As of: {as_of_date.date()}",
    legend_title_text="Label",
    xaxis_title="Time to Maturity (Years)",
    yaxis_title=f"YTMs: {quote_type}_yield",
    width=1400,
    height=800,
    template="plotly_dark",
)

fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)


fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikethickness=1)
fig.show()